# Comprehensive Guide To Approximate Nearest Neighbors Algorithms

## Exhaustive Search Usage

In [15]:
from annoy import AnnoyIndex
import nmslib
from pathlib import Path
import pandas as pd
import time

In [ ]:
# does not work/url does not exit
# import pickle
# import faiss
# def load_data():
#     with open('movies.pickle', 'rb') as f:
#         data = pickle.load(f)
#     return data
#     # return datadata = load_data()
# data = load_data()
# print (data)

## recrateing movie lesne directly


### Load and Preprocess the Movie Data
You need to prepare your movie data for encoding. For example, you may want to combine the title, description, and genres to form a rich text representation of each movie.

In [2]:
# import pandas as pd

# Path to the movie metadata file
# movies_file_path = 'ml-100k/u.item'
movies_file_path = Path('./MovieLens/ml-100k/u.item')
# Define all 24 column names based on the dataset description
movie_columns = ['movie_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL'] + \
                [f'genre_{i}' for i in range(19)]  # 19 genre columns



# Load the data with the correct number of columns
movies = pd.read_csv(movies_file_path, sep='|', encoding='latin-1', header=None, names=movie_columns)

# Display the first few rows
print(movies.head())


   movie_id              title release_date  video_release_date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDb_URL  genre_0  genre_1  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0        0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0        1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0        0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0        1   
4  http://us.imdb.com/M/title-exact?Copycat%20(1995)        0        0   

   genre_2  genre_3  genre_4  ...  genre_9  genre_10  genre_11  genre_12  \
0        0        1        1  ...        0         0         0         0   


In [9]:
# import pandas as pd
# import pickle

# Load the MovieLens 100K ratings data
# ratings_file_path = 'ml-100k/u.data'  # Replace with the correct path to the 'u.data' file
ratings_file_path = Path('./MovieLens/ml-100k/u.data')

# The data does not have headers in the file, so we specify them
columns = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(ratings_file_path, sep='\t', names=columns)

# Display first few rows of the dataset
print(ratings.head())

# Save the ratings data as a pickle file
with open('movielens_100k_ratings.pickle', 'wb') as f:
    pickle.dump(ratings, f)

print("MovieLens 100K ratings data saved to movielens_100k_ratings.pickle!")


   user_id  movie_id  rating  timestamp
0      196       242       3  881250949
1      186       302       3  891717742
2       22       377       1  878887116
3      244        51       2  880606923
4      166       346       1  886397596
MovieLens 100K ratings data saved to movielens_100k_ratings.pickle!


In [3]:
# Example: Create a text representation for each movie by combining title and genres
movies['text_representation'] = movies['title'] + ' ' + movies['release_date']

# Display the dataset with the text representation
print(movies[['movie_id', 'text_representation']].head())

   movie_id            text_representation
0         1   Toy Story (1995) 01-Jan-1995
1         2   GoldenEye (1995) 01-Jan-1995
2         3  Four Rooms (1995) 01-Jan-1995
3         4  Get Shorty (1995) 01-Jan-1995
4         5     Copycat (1995) 01-Jan-1995


### Use a Pretrained Model to Generate Embeddings
Now, we’ll use a pretrained transformer model to create semantic embeddings for the movie text data. The model will convert the text representation of each movie into a vector of fixed size, capturing its semantic meaning.

In [6]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained model (e.g., 'all-MiniLM-L6-v2' is small and fast)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate semantic embeddings for the movie text representations
movie_descriptions = movies['text_representation'].tolist()
movie_embeddings = model.encode(movie_descriptions, show_progress_bar=True)

# The movie_embeddings is a matrix where each row is the vector for a movie
print(f"Shape of embeddings: {movie_embeddings.shape}")


c:\Users\Hassan\anaconda3\envs\Py39Stanford1\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Hassan\anaconda3\envs\Py39Stanford1\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Hassan\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/53 [00:00<?, ?it/s]

c:\Users\Hassan\anaconda3\envs\Py39Stanford1\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Shape of embeddings: (1682, 384)


### Save the Encoded Data
You can now save these embeddings for future use (e.g., for similarity search or as input to a recommendation model). You might also want to save the movie titles along with the embeddings to keep track of which vector corresponds to which movie.

In [16]:
import pickle

# Save the movie embeddings and their corresponding titles
with open('movie_embeddings.pickle', 'wb') as f:
    pickle.dump((movies['movie_id'].tolist(), movie_embeddings), f)

print("Movie embeddings saved to 'movie_embeddings.pickle'")


Movie embeddings saved to 'movie_embeddings.pickle'


### Perform Similarity Search (Optional)
If your goal is to perform a similarity search (e.g., find similar movies based on their semantic embeddings), you can use FAISS, a library that allows efficient similarity searches over large vector spaces.

In [9]:
import faiss
import numpy as np

# Convert embeddings to a NumPy array
movie_embeddings_np = np.array(movie_embeddings)

# Create a FAISS index
index = faiss.IndexFlatL2(movie_embeddings_np.shape[1])  # L2 distance (Euclidean)
index.add(movie_embeddings_np)  # Add embeddings to the index

# Search for the 5 most similar movies to the first movie in the dataset
query_embedding = movie_embeddings_np[0].reshape(1, -1)
distances, indices = index.search(query_embedding, 5)

# Print the most similar movies
for idx in indices[0]:
    print(f"Movie ID: {movies.iloc[idx]['movie_id']}, Title: {movies.iloc[idx]['title']}")


Movie ID: 1, Title: Toy Story (1995)
Movie ID: 1219, Title: Goofy Movie, A (1995)
Movie ID: 1470, Title: Gumby: The Movie (1995)
Movie ID: 772, Title: Kids (1995)
Movie ID: 93, Title: Welcome to the Dollhouse (1995)


In [10]:
text_representation_array = np.array(movies['text_representation'])

### copy paper

In [37]:
class ExactIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        # self.lab
        self.labels = labels
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension,)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [41]:
start_time = time.time()
movie_desc = np.array(movies['text_representation'])
index = ExactIndex(movie_embeddings_np, movie_desc)
index.build()
end_time = time.time()
print(f'Time to build with IndexFlatL2 {end_time - start_time}')

Time to build with IndexFlatL2 0.0039975643157958984


In [44]:
# index.query(data['vector'][0])
start_time = time.time()
index.query(movie_embeddings_np[0].reshape(1, -1))
# movie_embeddings_np[0]
end_time = time.time()
print(f'Time to query with IndexFlatL2 {end_time - start_time}')

Time to query with IndexFlatL2 0.000997781753540039


### ANN implementaion

In [31]:
from annoy import AnnoyIndex


In [39]:
# # class AnnoyIndex():
# #     def __init__(self, vectors, labels):
# #         self.dimension = vectors.shape[1]
# #         self.vectors = vectors.astype('float32')
# #         self.labels = labels    
   
# #     def build(self, number_of_trees=5):
# #         self.index = annoy.AnnoyIndex(self.dimension)
# #         for i, vec in enumerate(self.vectors):
# #             self.index.add_item(i, vec.tolist())
# #         self.index.build(number_of_trees)
        
# #     def query(self, vector, k=10):
# #         indices = self.index.get_nns_by_vector(
# #               vector.tolist(), 
# #               k, 
# #               search_k=search_in_x_trees)                                           
# #         return [self.labels[i] for i in indices]

# from annoy import AnnoyIndex

# class AnnoyIndexWrapper():
#     def __init__(self, vectors, labels):
#         self.dimension = vectors.shape[1]
#         self.vectors = vectors.astype('float32')
#         self.labels = labels    
   
#     def build(self, number_of_trees=5):
#         self.index = AnnoyIndex(self.dimension)
#         for i, vec in enumerate(self.vectors):
#             self.index.add_item(i, vec.tolist())
#         self.index.build(number_of_trees)
        
#     def query(self, vector, k=10):
#         indices = self.index.get_nns_by_vector(
#             vector.tolist(), 
#             k, 
#             search_k=search_in_x_trees
#         )                                           
#         return [self.labels[i] for i in indices]


In [ ]:
# ann_index = AnnoyIndex(movie_embeddings_np, movie_desc)
# ann_index.build()

In [27]:
class AnnoyIndexWrapper():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels
    
    def build(self, number_of_trees=5):
        self.index = AnnoyIndex(self.dimension, 'angular')  # Added metric type
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
    
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(
            vector.tolist(), 
            k, 
            search_k=-1
        )
        return [self.labels[i] for i in indices]


In [32]:
start_time = time.time()
ann_index = AnnoyIndexWrapper(movie_embeddings_np, movie_desc)
ann_index.build()
end_time = time.time()
print(f'Time to build with AnnoyIndex {end_time - start_time}')

Time to build with AnnoyIndex 0.05190420150756836


In [33]:
# index.query(data['vector'][0])
# ann_index.query(movie_embeddings_np[0].reshape(1, -1))
start_time = time.time()
query_vector = movie_embeddings_np[0].reshape(-1) 
ann_index.query(query_vector)
end_time = time.time()
print(f'Time to query with AnnoyIndex {end_time - start_time}')

Time to query with AnnoyIndex 0.000989675521850586


### Vector Encoding Using LSH

In [25]:
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimension, num_bits)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [23]:
start_time = time.time()
index_lsh = LSHIndex(movie_embeddings_np, movie_desc)
index_lsh.build()
end_time = time.time()
print(f'Time to build with LSHIndex {end_time - start_time}')

Time to build with LSHIndex 0.0029926300048828125


In [26]:
# query_vector.reshape(1, -1)
start_time = time.time()
movie_embeddings_np[0]
index_lsh.query(movie_embeddings_np[0].reshape(1, -1))
end_time = time.time()
print(f'Time to search with LSHIndex {end_time - start_time}')

Time to search with LSHIndex 0.0009503364562988281


### Vector Encoding Using Quantization

In [19]:
class IVPQIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
    
    def build(self, 
              number_of_partition=8, 
              search_in_x_partitions=2, 
              subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimension, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [20]:
start_time = time.time()
Quant_index = IVPQIndex(movie_embeddings_np, movie_desc)
Quant_index.build()
end_time = time.time()
print(f'Time to build with Quant_index {end_time - start_time}')

Time to build with Quant_index 0.1565852165222168


In [21]:
start_time = time.time()
Quant_index.query(movie_embeddings_np[0].reshape(1, -1))
end_time = time.time()
print(f'Time to search  with Quant_index {end_time - start_time}')

Time to search  with Quant_index 0.0009963512420654297


### Hierarchical Navigable Small World Graphs

In [11]:
class NMSLIBIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels
    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

In [16]:
import nmslib
movie_desc = np.array(movies['text_representation'])
start_time = time.time()
NSM_index = NMSLIBIndex(movie_embeddings_np, movie_desc)
NSM_index.build()
end_time = time.time()
print(f'Time to build with nmslib {end_time - start_time}')

Time to build with nmslib 0.14765191078186035


In [18]:
start_time = time.time()
print (f'search {NSM_index.query(movie_embeddings_np[0].reshape(1, -1))}')
end_time = time.time()
print(f'Time to search with nmslib {end_time - start_time}')

search ['Toy Story (1995) 01-Jan-1995', 'Goofy Movie, A (1995) 01-Jan-1995', 'Gumby: The Movie (1995) 01-Jan-1995', 'Kids (1995) 01-Jan-1995', 'Welcome to the Dollhouse (1995) 24-May-1996', 'Little Princess, A (1995) 01-Jan-1995', 'Candyman: Farewell to the Flesh (1995) 01-Jan-1995', 'Castle Freak (1995) 01-Jan-1995', 'Boys Life (1995) 01-Jan-1995', 'Friday (1995) 01-Jan-1995']
Time to search with nmslib 0.0009999275207519531
